 # 1. DEFINE I/O DATA FRAMES

In [1]:
import numpy as np 
import pandas as pd 

MODEL_NAME='multi_bibasic_lstm'
HPARAMS_STR='lr5.00e-04,hs128x64'
MODEL_V= '12'
INPUT_V='00'
INPUT_DIR= '../datasets/csvs/'
OUTPUT_DIR='../outputs/'

INPUT_PATH = '{:}{:}'.format(INPUT_DIR, 'zhou_valid.csv')
OUTPUT_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'Yhat_valid.csv')
TARGET_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'evaluation_valid.csv')



print(INPUT_PATH)
print(OUTPUT_PATH)

../datasets/csvs/zhou_valid.csv
../outputs/multi_bibasic_lstm/lr5.00e-04,hs128x64/12/Yhat_valid.csv


In [2]:
df_input= pd.read_csv(INPUT_PATH, index_col=0)
df_output=pd.read_csv(OUTPUT_PATH, index_col=0)
print(df_input.shape, df_output.shape)

(12298, 10) (12298, 1)


In [3]:
df= df_input.join(df_output, how='left')
print(df.shape)

(12298, 11)


In [4]:
df.to_csv(TARGET_PATH)

In [5]:
df.head()

,ID,S,P,P_S,FORM,LEMMA,PRED,M_R,ARG,ARG_Y,Yhat
IDX,,,,,,,,,,,
123846,1,2879,1,5100,Entre,entre,estar,0,(A2*,A2,A2
123847,2,2879,1,5100,os,o,estar,0,*,A2,A2
123848,3,2879,1,5100,convidados,convidado,estar,0,*),A2,A1
123849,4,2879,1,5100,estão,estar,estar,1,(V*),V,V
123850,5,2879,1,5100,o,o,estar,1,(A1*,A1,A2


In [6]:
df2= df[['ARG_Y','Yhat']]
table= pd.pivot_table(df2.iloc[:30,:], index='ARG_Y', columns='Yhat', aggfunc=len, fill_value=0)
table

Yhat,*,A0,A1,A2,V
ARG_Y,,,,,
*,1,0,0,0,0
A0,0,1,0,0,0
A1,0,0,22,2,0
A2,0,0,1,2,0
V,0,0,0,0,1


 # 2. CONFUSION MATRIX

In [58]:
df1= df[['ARG_Y','Yhat']]
table= pd.pivot_table(df1, index='ARG_Y', columns='Yhat', aggfunc=len, fill_value=0)

#Makes a unique index out of ground truth and predictions, filling with zeros
new_index= table.index.union(table.columns)
table= table.reindex(new_index).fillna(0)
table.columns.names=['predictions']
table.index.name='ground truth'
table


predictions,*,A0,A1,A2,A3,A4,AM-ADV,AM-CAU,AM-DIR,AM-DIS,AM-EXT,AM-LOC,AM-MED,AM-MNR,AM-NEG,AM-PNC,AM-PRD,AM-REC,AM-TMP,V
ground truth,,,,,,,,,,,,,,,,,,,,
*,3645.0,154.0,204.0,97.0,4.0,11.0,31.0,1.0,0.0,2.0,0.0,19.0,0.0,11.0,4.0,1.0,20.0,0.0,17.0,22.0
A0,356.0,454.0,75.0,32.0,1.0,0.0,9.0,10.0,0.0,4.0,2.0,7.0,0.0,12.0,1.0,0.0,4.0,0.0,10.0,0.0
A1,711.0,103.0,1735.0,273.0,14.0,4.0,49.0,17.0,0.0,4.0,11.0,52.0,0.0,41.0,13.0,20.0,21.0,3.0,42.0,16.0
A2,369.0,51.0,346.0,508.0,0.0,6.0,2.0,3.0,4.0,0.0,0.0,5.0,0.0,6.0,0.0,4.0,9.0,1.0,12.0,23.0
A3,6.0,2.0,13.0,17.0,12.0,6.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,12.0,0.0,0.0,2.0,0.0,10.0,0.0
A4,2.0,0.0,4.0,9.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
AM-ADV,34.0,8.0,42.0,4.0,1.0,0.0,12.0,1.0,0.0,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AM-CAU,71.0,13.0,28.0,7.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,1.0,4.0,3.0,0.0,0.0,5.0,0.0,6.0,1.0
AM-DIR,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


 ## 3 Evaluation
 ### Precision .: How many selected items are relevant?
 ### Recall .: How many relevant items are selected?

In [65]:
# BASE MATRICES
diag = np.diag(table.as_matrix()).astype(np.float32)
retrieved= np.sum(table.as_matrix(), axis=0, keepdims=False).astype(np.float32) #preserve columns, sum over rows
relevant= np.sum(table.as_matrix(), axis=1, keepdims=False).astype(np.float32) #preserve rows, sum over columns

#PRECISION
P=np.nan_to_num(diag / retrieved)
precisiondf= pd.DataFrame(data= P, index=table.index, columns=['precision']).fillna(0)
precisiondf.index.name= 'ARG'
# precisiondf

#RECALL
R= np.nan_to_num(diag / relevant)
recalldf= pd.DataFrame(data= R, index=table.index, columns=['recall']).fillna(0)
recalldf.index.name= 'ARG'
# recalldf

#F1
F1= np.nan_to_num(2* ((P * R)/(P + R)))
f1scoredf= pd.DataFrame(data= F1, index=table.index, columns=['f1 score']).fillna(0)
f1scoredf.index.name= 'ARG'

#Concatenate
evaluationdf= pd.concat((precisiondf, recalldf, f1scoredf), axis=1)
evaluationdf








/Users/guilhermevarela/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/Users/guilhermevarela/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


,precision,recall,f1 score
ARG,,,
*,0.628123,0.859062,0.725662
A0,0.507830,0.464688,0.485302
A1,0.634137,0.554490,0.591645
A2,0.442895,0.376575,0.407051
A3,0.266667,0.144578,0.187500
A4,0.377358,0.444444,0.408163
AM-ADV,0.097561,0.108108,0.102564
AM-CAU,0.202899,0.091503,0.126126
AM-DIR,0.000000,0.000000,0.000000
